Importing applicationInformation

In [1]:
import pandas as pd
import datetime
pd.set_option('display.max_column', None)
game_data = pd.read_csv('applicationInformation.csv', 
                        encoding = 'latin-1',
                    )
#Converting freetoplay to boolean
game_data.freetoplay = game_data.freetoplay.replace({1.0: True, 0.0: False}) 


In [2]:
#Cleaning Colnames
game_data.rename(columns = {'appid': 'app_id',
                           'releasedate' : 'release_date',
                           'freetoplay' : 'free_to_play'},
                            inplace = True)



#release_date to datetime
game_data['release_date'] = pd.to_datetime(game_data['release_date'],format = '%d-%b-%y', errors = 'coerce')



Combining with Application Developers

In [3]:
game_devs = pd.read_csv('applicationDevelopers.csv',
                        names = ['app_id', 'developer', 'developer_2', 'developer_3'],
                        encoding = 'latin-1')
game_data = pd.merge(game_data, game_devs, on = 'app_id', how = 'inner')
game_data

,app_id,type,name,release_date,free_to_play,developer,developer_2,developer_3
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN
...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN


Combining w/ Publishers

In [4]:
game_pub = pd.read_csv('applicationPublishers.csv',
                      names = ['app_id', 'publisher_1', 'publisher_2', 'publisher_3'],
                      encoding = 'latin-1'
                      )
game_data = pd.merge(game_data, game_pub, on = 'app_id', how = 'inner')
game_data

,app_id,type,name,release_date,free_to_play,developer,developer_2,developer_3,publisher_1,publisher_2,publisher_3
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN,SEGA,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN,Greens s.r.o.,NaN,NaN
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN,Activision,NaN,NaN


One Hot Encoding Genres

In [5]:
#Reading and setting up short df
game_genres_long = pd.read_csv('applicationGenres.csv',
                      names = ['app_id', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6'],
                      encoding = 'latin-1',
                      index_col = False
                      )
game_genres = pd.DataFrame(columns = ['app_id', 'genre'])
for index, row in game_genres_long.iterrows():
    for c in range(1,7):
        if(pd.isnull(row[c])):
            c = 8
        else:
            app_id = row[0] #Getting appid
            genre = row[c] #Getting genre
            game_genres.loc[len(game_genres.index)] = [app_id, genre] #Adding to short df 
            


In [6]:
#One-Hot Encoding
game_genres_final = pd.get_dummies(game_genres, columns = ['genre'], prefix='genre')
game_genres_final.drop('genre_60', axis = 1, inplace = True)
game_genres_final = game_genres_final.groupby('app_id', sort = False).sum()
game_genres_final

,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing
app_id,,,,,,,,,,,,,,,,,,,,,,,,,
578080,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
570,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
730,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
359550,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
271590,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
227860,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
429050,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [7]:
#Getting genre names
col_names = list(game_genres_final)

In [8]:
#Merging into master df
game_data = pd.merge(game_data, game_genres_final, on = 'app_id', how = 'left')
game_data[col_names] = game_data[col_names].replace({1.0: True, 0.0: False}) #Convert to True/False
game_data

,app_id,type,name,release_date,free_to_play,developer,developer_2,developer_3,publisher_1,publisher_2,publisher_3,genre_Action,genre_Adventure,genre_Animation & Modeling,genre_Audio Production,genre_Casual,genre_Design & Illustration,genre_Early Access,genre_Education,genre_Free to Play,genre_Gore,genre_Indie,genre_Massively Multiplayer,genre_Nudity,genre_Photo Editing,genre_RPG,genre_Racing,genre_Sexual Content,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Violent,genre_Web Publishing
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,False,PUBG Corporation,NaN,NaN,PUBG Corporation,NaN,NaN,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,570,game,Dota 2,2013-07-09,True,Valve,NaN,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,730,game,Counter-Strike: Global Offensive,2012-08-21,True,Valve,Hidden Path Entertainment,NaN,Valve,NaN,NaN,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,2015-12-01,False,Ubisoft Montreal,NaN,NaN,Ubisoft,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,2009-10-29,True,Sports Interactive,NaN,NaN,SEGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,202480,NaN,Skyrim Creation Kit,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,429050,game,Feed and Grow: Fish,2016-01-08,False,Old B1ood,NaN,NaN,Greens s.r.o.,NaN,NaN,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1998,209650,game,Call of Duty: Advanced Warfare,2014-11-03,False,Sledgehammer Games,Raven Software,NaN,Activision,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Exporting to csv

In [9]:
game_data.to_csv('cleaned_game_data.csv')